In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data_hpcl_all = pd.read_csv("HPCL_books_data.csv")

In [3]:
data_nearest_3_years = data_hpcl_all[data_hpcl_all["publish_year"].isin(["[2024]", "[2023]", "[2022]"])]

In [4]:
data_nearest_3_years.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7309 entries, 149 to 8748
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    7309 non-null   object 
 1   ddc_sort                 3 non-null      object 
 2   ratings_average          75 non-null     float64
 3   ebook_count_i            7309 non-null   int64  
 4   edition_count            7309 non-null   int64  
 5   key                      7309 non-null   object 
 6   publisher                7309 non-null   object 
 7   publish_year             7309 non-null   object 
 8   type                     7309 non-null   object 
 9   isbn                     7276 non-null   object 
 10  author_name              7249 non-null   object 
 11  ratings_count_1          75 non-null     float64
 12  ratings_count_2          75 non-null     float64
 13  author_alternative_name  635 non-null    object 
 14  seed                  

In [11]:
data_nearest_3_years.to_csv("HPCL_nearest_3_years.csv", index=False)

In [5]:
data_10000_samples = pd.read_csv("HarperCollins_10000_sample.csv")

In [6]:
data_scraped = pd.read_csv("data_0503/HarperCollins_Amazon.csv", encoding="latin-1")

In [7]:
data_scraped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23652 entries, 0 to 23651
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   all_rank_list        21085 non-null  object
 1   format               23652 non-null  object
 2   before_discount      23652 non-null  object
 3   new_price            23652 non-null  object
 4   title                23652 non-null  object
 5   title_search_result  23652 non-null  object
 6   other_list           23404 non-null  object
dtypes: object(7)
memory usage: 1.3+ MB


In [8]:
data_scraped_valid = data_scraped[data_scraped["other_list"].fillna("").str.contains("HarperCollins")]

In [9]:
data_scraped_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2914 entries, 17 to 23639
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   all_rank_list        2409 non-null   object
 1   format               2914 non-null   object
 2   before_discount      2914 non-null   object
 3   new_price            2914 non-null   object
 4   title                2914 non-null   object
 5   title_search_result  2914 non-null   object
 6   other_list           2914 non-null   object
dtypes: object(7)
memory usage: 182.1+ KB


In [39]:
data_scraped_valid["other_list"].to_csv("xxx.csv")

In [10]:
data_scraped_invalid = data_scraped[~data_scraped["other_list"].fillna("").str.contains("HarperCollins")]

In [20]:
def extract_publish_year(text):
    match = re.search(r"\b(\d{4})\b", str(text))
    if match:
        return int(match.group(1))
    return None


data_scraped_invalid["publish_year"] = data_scraped_invalid["other_list"].apply(extract_publish_year)
data_scraped_valid["publish_year"] = data_scraped_valid["other_list"].apply(extract_publish_year)
data_scraped["publish_year"] = data_scraped["other_list"].apply(extract_publish_year)

C:\Users\USER\AppData\Local\Temp\ipykernel_19408\1635963699.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\USER\AppData\Local\Temp\ipykernel_19408\1635963699.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [40]:
extract_publish_year(
    "Publisher:HarperCollins Hamburg (November 30, 2021)   Language:German   Paperback:429 pages   ISBN-10:3749904081   ISBN-13:978-3749904082   Item Weight:12.7 ounces   Dimensions:4.92 x 1.18 x 7.32 inches  "
)

2021

In [16]:
data_scraped_valid.info()
data_scraped_invalid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2914 entries, 17 to 23639
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   all_rank_list        2409 non-null   object
 1   format               2914 non-null   object
 2   before_discount      2914 non-null   object
 3   new_price            2914 non-null   object
 4   title                2914 non-null   object
 5   title_search_result  2914 non-null   object
 6   other_list           2914 non-null   object
 7   publish_year         2914 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 204.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20738 entries, 0 to 23651
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   all_rank_list        18676 non-null  object 
 1   format               20738 non-null  object 
 2   before_discount      20738 non-null  

In [17]:
import plotly.graph_objects as go

# 計算IQR
Q1 = data_scraped_invalid["publish_year"].quantile(0.25)
Q3 = data_scraped_invalid["publish_year"].quantile(0.75)
IQR = Q3 - Q1

# 定義極端值的範圍
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 過濾掉極端值
filtered_data = data_scraped_invalid[(data_scraped_invalid["publish_year"] >= lower_bound) & (data_scraped_invalid["publish_year"] <= upper_bound)]

# 繪製直方圖
fig = go.Figure(data=[go.Histogram(x=filtered_data["publish_year"])])
fig.update_layout(title_text="[Invalid] Histogram of publish_year", xaxis_title="publish_year", yaxis_title="Frequency")
fig.show()

In [18]:
import plotly.graph_objects as go

# 計算IQR
Q1 = data_scraped_valid["publish_year"].quantile(0.25)
Q3 = data_scraped_valid["publish_year"].quantile(0.75)
IQR = Q3 - Q1

# 定義極端值的範圍
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 過濾掉極端值
filtered_data = data_scraped_valid[(data_scraped_valid["publish_year"] >= lower_bound) & (data_scraped_valid["publish_year"] <= upper_bound)]

# 繪製直方圖
fig = go.Figure(data=[go.Histogram(x=filtered_data["publish_year"])])
fig.update_layout(title_text="[Valid] Histogram of publish_year", xaxis_title="publish_year", yaxis_title="Frequency")
fig.show()

In [27]:
import plotly.graph_objects as go
import pandas as pd

# 計算每個年份在data_scraped_invalid和data_scraped中的數量
invalid_counts = data_scraped_invalid["publish_year"].value_counts()
total_counts = data_scraped["publish_year"].value_counts()

# 計算比例
invalid_ratios = invalid_counts / total_counts

# 創建一個新的DataFrame來存儲年份和比例
df = pd.DataFrame({"Year": invalid_ratios.index, "Ratio": invalid_ratios.values})

# 繪製直方圖
fig = go.Figure(data=[go.Bar(x=df["Year"], y=df["Ratio"])])
fig.update_layout(title_text="Histogram of publish_year", xaxis_title="publish_year", yaxis_title="Ratio of invalid")
fig.update_xaxes(range=[1990, 2024.5])  # 更新 x 軸的範圍
fig.show()